# Real Data Analysis

In this section, we import necessary libraries and tools to perform real data analysis. 

- **Libraries and Tools:**
  - `final_util`: A custom utility module for specific model functions.
  - `numpy`: For numerical computations and array manipulations.
  - `pandas`: For handling and preprocessing tabular data.
  - `os`: For interacting with the operating system to manage files and directories.
  - `sklearn.preprocessing.StandardScaler`: For standardizing features by removing the mean and scaling to unit variance.
  - `matplotlib.pyplot`: For data visualization and plotting.
  - `sklearn.model_selection.KFold`: For implementing k-fold cross-validation to evaluate model performance. 

In [2]:
from final_util import * 
import numpy as np
import pandas as pd
import os 
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

## Low-Dimensional Real Dataset

In this section, we analyze a low-dimensional real-world dataset: **housing price data**. This dataset contains information on housing features and their corresponding prices, making it suitable for regression analysis in a low-dimensional setting.

- **Dataset Overview**:
  - The dataset includes various features related to houses, such as median income,housing median age,total population,number of households,and the total number of rooms across all houses within a block group.
  - The target variable is the logarithm of the median house value of each block group.

- **Download Address**:
  The dataset can be downloaded from the following link: [Housing Price Data](http://lib.stat.cmu.edu/datasets/houses.zip) 


### Data Preparation

In this step, we process the housing price dataset for analysis:

1. **Data Loading**:
   - Read the dataset from the  local file.
   - The dataset used in this analysis is derived from the original **cadata.txt** file. The **cadata_new.txt** file has been preprocessed by removing the textual descriptions from the original dataset, leaving only the numerical data.   
   - Extract the predictor variables (features) and response variable (median house value).

2. **Log Transformation**:
   - Apply a logarithmic transformation to the response variable (median house value).

3. **Feature Standardization**:
   - Standardize the predictor variables.


In [3]:
### data ###
current_dir = os.getcwd()
file_path = os.path.join(current_dir, 'realdata/lowdim_house_data/cadata_new.txt')
data_org  = np.loadtxt(file_path)
data_house_org = pd.DataFrame(data_org,columns=['median_house_value','median_income','housing_median_age','total_rooms','total_bedrooms','population','households','latitude','longitude'])
data_house = data_house_org[['median_house_value','median_income','housing_median_age','population','households','total_rooms']]

Y_house = np.array(np.log(data_house['median_house_value']))
X_house = data_house.iloc[:,1:] # p=5
## standardize the data
scaler = StandardScaler()
X_house = pd.DataFrame(scaler.fit_transform(X_house),columns=X_house.columns)

###  Compare the ε-GDP Huber vs. Non-Private Huber

In this section, we compare the performance of `noisygd` with `gd`. The ε-GDP  Huber regression introduces privacy-preserving noise into the model, while the non-private Huber regression does not.

We will evaluate both models in terms of:
1. **Error comparison**: Measure the prediction errors for both models using the same dataset.
2. **Impact of privacy**: Analyze how the introduction of noise in the ε-GDP Huber regression affects the model's performance, particularly as the privacy parameter (ε) varies, in comparison to the benchmark, which is the performance of the `gd` applied to the entire dataset without any privacy constraints.


In [4]:
sequence = np.arange(400, 20001, 400)
Y_house_copy = np.array(Y_house.copy()).reshape(-1)
X_house_copy = np.array(X_house.copy())
n = X_house_copy.shape[0]
p = X_house_copy.shape[1]
 
#### Comparison ####
mu = np.array([.3, .5, .9])
result_mse = []
result_priv1_mse = []
result_priv2_mse = []
result_priv3_mse = []

for index in range(len(sequence)):
    rgt.seed(index+1)
    numm = sequence[index]
    print(numm)
    random_rows = np.random.choice(n, numm, replace=False)
    X_sample = X_house_copy[random_rows] 
    Y_sample = Y_house_copy[random_rows] 

    B = (p + np.log(numm))**0.5
    T = int(np.ceil(np.log(numm)))
    result_mse_temp = []
    result_priv1_mse_temp = []
    result_priv2_mse_temp = []
    result_priv3_mse_temp = []
    
    for rep in range(300):
        rgt.seed(rep + 1)
        train_idx = np.random.choice(numm, size=int(0.8 * numm), replace=False)
        test_idx = np.setdiff1d(np.arange(numm), train_idx)
        
        # Split into 80% training and 20% testing 
        X_train, X_test = X_sample[train_idx], X_sample[test_idx]
        Y_train, Y_test = Y_sample[train_idx], Y_sample[test_idx]
        
        X_test_new = np.concatenate([np.ones((len(X_test), 1)), X_test], axis=1)
        ## Non-private Huber
        train_size = 0.8 * numm
        model_huber_temp = Huber(X_train, Y_train)
        out_huber_temp = model_huber_temp.gd(robust=.5 * (train_size/(p+np.log(train_size)))**0.5, lr=.5, max_niter=T)
        Y_pred = X_test_new @ out_huber_temp['beta']
        mse_huber = np.mean((Y_test - Y_pred) ** 2)
        
        ## ε-GDP Huber
        priv_robust = .5 * mad(out_huber_temp['residuals']) * (train_size * mu / (p + np.log(train_size)))**0.5
        out_priv1_temp = model_huber_temp.noisygd(priv_robust[0], lr=.5, B=B, mu=mu[0], T=T)
        out_priv2_temp = model_huber_temp.noisygd(priv_robust[1], lr=.5, B=B, mu=mu[1], T=T)
        out_priv3_temp = model_huber_temp.noisygd(priv_robust[2], lr=.5, B=B, mu=mu[2], T=T)
        
        # Compute MSE for private versions
        Y_priv1_pred = X_test_new @ out_priv1_temp['beta']
        Y_priv2_pred = X_test_new @ out_priv2_temp['beta']
        Y_priv3_pred = X_test_new @ out_priv3_temp['beta']
        
        mse_priv1 = np.mean((Y_test - Y_priv1_pred) ** 2)
        mse_priv2 = np.mean((Y_test - Y_priv2_pred) ** 2)
        mse_priv3 = np.mean((Y_test - Y_priv3_pred) ** 2)
        
        result_mse_temp.append(mse_huber)
        result_priv1_mse_temp.append(mse_priv1)
        result_priv2_mse_temp.append(mse_priv2)
        result_priv3_mse_temp.append(mse_priv3)
    
    result_mse.append(np.mean(result_mse_temp))
    result_priv1_mse.append(np.mean(result_priv1_mse_temp))
    result_priv2_mse.append(np.mean(result_priv2_mse_temp))
    result_priv3_mse.append(np.mean(result_priv3_mse_temp))


400
800
1200
1600
2000
2400
2800
3200
3600
4000
4400
4800
5200
5600
6000
6400
6800
7200
7600
8000
8400
8800
9200
9600
10000
10400
10800
11200
11600
12000
12400
12800
13200
13600
14000
14400
14800
15200
15600
16000
16400
16800
17200
17600
18000
18400
18800
19200
19600
20000


## High-Dimensional Real Dataset

In this section, we analyze a high-dimensional real-world dataset: **Communities and Crime**. This dataset contains information about various features of communities and their corresponding crime rates.

- **Dataset Overview**:
  - The dataset includes socio-economic and crime data from U.S. communities.  There are 125 features with 18 potential target variables (e.g., various crime rates). 
  - The target variable in our analysis is the logarithm of **ViolentCrimesPerPop**(total number of violent crimes per 100,000 population).

- **Download Address**:
  The dataset can be downloaded from the following link: [Communities and Crime Dataset](https://archive.ics.uci.edu/dataset/211/communities+and+crime+unnormalized) 


### Data Preprocessing for Communities and Crime Dataset

In this step, we process the dataset by selecting **ViolentCrimesPerPop** as the response variable. We handle multicollinearity by removing the following variables, which are linearly correlated:

- `pctUrban`: Percentage of urban population.
- `OwnOccQrange`: Difference between the upper and lower quartiles of home ownership.
- `RentQrange`: Difference between high and low rent quartiles.

After removing these correlated variables, we delete columns with missing data and remove rows where the response variable is missing, leaving 1994 rows and 99 columns for further analysis.

In [6]:
### data ###
current_dir = os.getcwd()
file_path = os.path.join(current_dir, 'realdata/highdim/Data.txt')
data_org = pd.read_csv(file_path, delimiter=',', header=None)
colnames_comm = [
    "communityname", "state", "countyCode", "communityCode", "fold", "population", "householdsize",
    "racepctblack", "racePctWhite", "racePctAsian", "racePctHisp", "agePct12t21", "agePct12t29",
    "agePct16t24", "agePct65up", "numbUrban", "pctUrban", "medIncome", "pctWWage", "pctWFarmSelf",
    "pctWInvInc", "pctWSocSec", "pctWPubAsst", "pctWRetire", "medFamInc", "perCapInc", "whitePerCap",
    "blackPerCap", "indianPerCap", "AsianPerCap", "OtherPerCap", "HispPerCap", "NumUnderPov",
    "PctPopUnderPov", "PctLess9thGrade", "PctNotHSGrad", "PctBSorMore", "PctUnemployed", "PctEmploy",
    "PctEmplManu", "PctEmplProfServ", "PctOccupManu", "PctOccupMgmtProf", "MalePctDivorce",
    "MalePctNevMarr", "FemalePctDiv", "TotalPctDiv", "PersPerFam", "PctFam2Par", "PctKids2Par",
    "PctYoungKids2Par", "PctTeen2Par", "PctWorkMomYoungKids", "PctWorkMom", "NumKidsBornNeverMar",
    "PctKidsBornNeverMar", "NumImmig", "PctImmigRecent", "PctImmigRec5", "PctImmigRec8", "PctImmigRec10",
    "PctRecentImmig", "PctRecImmig5", "PctRecImmig8", "PctRecImmig10", "PctSpeakEnglOnly",
    "PctNotSpeakEnglWell", "PctLargHouseFam", "PctLargHouseOccup", "PersPerOccupHous", "PersPerOwnOccHous",
    "PersPerRentOccHous", "PctPersOwnOccup", "PctPersDenseHous", "PctHousLess3BR", "MedNumBR", "HousVacant",
    "PctHousOccup", "PctHousOwnOcc", "PctVacantBoarded", "PctVacMore6Mos", "MedYrHousBuilt", "PctHousNoPhone",
    "PctWOFullPlumb", "OwnOccLowQuart", "OwnOccMedVal", "OwnOccHiQuart", "OwnOccQrange", "RentLowQ",
    "RentMedian", "RentHighQ", "RentQrange", "MedRent", "MedRentPctHousInc", "MedOwnCostPctInc",
    "MedOwnCostPctIncNoMtg", "NumInShelters", "NumStreet", "PctForeignBorn", "PctBornSameState",
    "PctSameHouse85", "PctSameCity85", "PctSameState85", "LemasSwornFT", "LemasSwFTPerPop",
    "LemasSwFTFieldOps", "LemasSwFTFieldPerPop", "LemasTotalReq", "LemasTotReqPerPop", "PolicReqPerOffic",
    "PolicPerPop", "RacialMatchCommPol", "PctPolicWhite", "PctPolicBlack", "PctPolicHisp", "PctPolicAsian",
    "PctPolicMinor", "OfficAssgnDrugUnits", "NumKindsDrugsSeiz", "PolicAveOTWorked", "LandArea", "PopDens",
    "PctUsePubTrans", "PolicCars", "PolicOperBudg", "LemasPctPolicOnPatr", "LemasGangUnitDeploy",
    "LemasPctOfficDrugUn", "PolicBudgPerPop", "murders", "murdPerPop", "rapes", "rapesPerPop",
    "robberies", "robbbPerPop", "assaults", "assaultPerPop", "burglaries", "burglPerPop", "larcenies",
    "larcPerPop", "autoTheft", "autoTheftPerPop", "arsons", "arsonsPerPop", "ViolentCrimesPerPop",
    "nonViolPerPop"
]
data_org.columns = colnames_comm
data_org_new = data_org[[  "fold", "population", "householdsize",
"racepctblack", "racePctWhite", "racePctAsian", "racePctHisp", "agePct12t21", "agePct12t29",
"agePct16t24", "agePct65up", "numbUrban", "pctUrban", "medIncome", "pctWWage", "pctWFarmSelf",
"pctWInvInc", "pctWSocSec", "pctWPubAsst", "pctWRetire", "medFamInc", "perCapInc", "whitePerCap",
"blackPerCap", "indianPerCap", "AsianPerCap", "OtherPerCap", "HispPerCap", "NumUnderPov",
"PctPopUnderPov", "PctLess9thGrade", "PctNotHSGrad", "PctBSorMore", "PctUnemployed", "PctEmploy",
"PctEmplManu", "PctEmplProfServ", "PctOccupManu", "PctOccupMgmtProf", "MalePctDivorce",
"MalePctNevMarr", "FemalePctDiv", "TotalPctDiv", "PersPerFam", "PctFam2Par", "PctKids2Par",
"PctYoungKids2Par", "PctTeen2Par", "PctWorkMomYoungKids", "PctWorkMom", "NumKidsBornNeverMar",
"PctKidsBornNeverMar", "NumImmig", "PctImmigRecent", "PctImmigRec5", "PctImmigRec8", "PctImmigRec10",
"PctRecentImmig", "PctRecImmig5", "PctRecImmig8", "PctRecImmig10", "PctSpeakEnglOnly",
"PctNotSpeakEnglWell", "PctLargHouseFam", "PctLargHouseOccup", "PersPerOccupHous", "PersPerOwnOccHous",
"PersPerRentOccHous", "PctPersOwnOccup", "PctPersDenseHous", "PctHousLess3BR", "MedNumBR", "HousVacant",
"PctHousOccup", "PctHousOwnOcc", "PctVacantBoarded", "PctVacMore6Mos", "MedYrHousBuilt", "PctHousNoPhone",
"PctWOFullPlumb", "OwnOccLowQuart", "OwnOccMedVal", "OwnOccHiQuart", "OwnOccQrange", "RentLowQ",
"RentMedian", "RentHighQ", "RentQrange", "MedRent", "MedRentPctHousInc", "MedOwnCostPctInc",
"MedOwnCostPctIncNoMtg", "NumInShelters", "NumStreet", "PctForeignBorn", "PctBornSameState",
"PctSameHouse85", "PctSameCity85", "PctSameState85", "LemasSwornFT", "LemasSwFTPerPop",
"LemasSwFTFieldOps", "LemasSwFTFieldPerPop", "LemasTotalReq", "LemasTotReqPerPop", "PolicReqPerOffic",
"PolicPerPop", "RacialMatchCommPol", "PctPolicWhite", "PctPolicBlack", "PctPolicHisp", "PctPolicAsian",
"PctPolicMinor", "OfficAssgnDrugUnits", "NumKindsDrugsSeiz", "PolicAveOTWorked", "LandArea", "PopDens",
"PctUsePubTrans", "PolicCars", "PolicOperBudg", "LemasPctPolicOnPatr", "LemasGangUnitDeploy",
"LemasPctOfficDrugUn", "PolicBudgPerPop",  "ViolentCrimesPerPop" ]]
#sum_ab = data_org_new['RentHighQ']-data_org_new['RentLowQ']- data_org_new['RentQrange'] ## all 0
multicollinearity = ['pctUrban','OwnOccQrange','RentQrange']
# pctUrban = 100*numbUrban / population
# OwnOccQrange = OwnOccHiQuart - OwnOccLowQuart
# RentQrange = RentHighQ - RentLowQ
missing_col = data_org_new.columns[data_org_new.apply(lambda col: col.astype(str).str.contains('\\?').any())]
missing_variable_name = ['OtherPerCap', 'LemasSwornFT', 'LemasSwFTPerPop', 'LemasSwFTFieldOps',
       'LemasSwFTFieldPerPop', 'LemasTotalReq', 'LemasTotReqPerPop',
       'PolicReqPerOffic', 'PolicPerPop', 'RacialMatchCommPol',
       'PctPolicWhite', 'PctPolicBlack', 'PctPolicHisp', 'PctPolicAsian',
       'PctPolicMinor', 'OfficAssgnDrugUnits', 'NumKindsDrugsSeiz',
       'PolicAveOTWorked', 'PolicCars', 'PolicOperBudg', 'LemasPctPolicOnPatr',
       'LemasGangUnitDeploy', 'PolicBudgPerPop']
## with missing values
 
columns_to_drop = multicollinearity + missing_variable_name
 
data_COMM = data_org_new.drop(columns=columns_to_drop)
data_COMM = data_COMM[data_COMM['ViolentCrimesPerPop'] != '?'] # Remove rows where the response variable is missing.
data_COMM = data_COMM.reset_index(drop=True)

Y_data_COMM = data_COMM['ViolentCrimesPerPop']
X_data_COMM = data_COMM.drop(columns='ViolentCrimesPerPop')

### Data Preprocessing: Standardization and Removal of Constant Variables

In this step:

1. **Standardization of the Response Variable**: The response variable **ViolentCrimesPerPop** is standardized using **StandardScaler**, which scales the variable to have zero mean and unit variance.

2. **Standardization of Covariates**: All covariates are also standardized using **StandardScaler** to ensure consistency across variables with different units of measurement.

3. **Removal of Intercept Term**: The intercept term is removed to facilitate comparison with the **sparse DP LS**.


In [7]:
from sklearn.preprocessing import StandardScaler
Y_data_COMM = Y_data_COMM.values.reshape(-1, 1) 
scaler = StandardScaler()
Y_COMM = scaler.fit_transform(Y_data_COMM) 
Y_COMM = pd.Series(Y_COMM.flatten()) 

scaler_x1 = StandardScaler()
X_COMM = pd.DataFrame(scaler_x1.fit_transform(X_data_COMM), columns=X_data_COMM.columns)
X_COMM = X_COMM.iloc[:,1:] 

### s Selection

To estimate the sparsity parameter **s**, we use the `noisygd_highdim` method with **10-fold Cross Validation**. The value of **s** is chosen by minimizing the **Mean Squared Error (MSE)**, ensuring that the sparsity level selected leads to the best predictive performance.



In [8]:
p = X_COMM.shape[1]
numm = 1994  
mu = 0.5 
delta = 0.01
lr = 0.1 
s_list = list(range(1, 11))
re_matrix = np.zeros((10, 1)) 
n_1 = 200 
n_2 = int(0.9*numm - n_1)
T = int(np.ceil(np.log(n_2)))  ## number of iterations
B_huber = .4 * (np.log(p) + np.log(n_2))**0.5  # truncation parameter
kf = KFold(n_splits=10, shuffle=True, random_state=42)  # 10-fold cross-validation

for s in s_list:
    print(f"Processing s = {s}")
    rgt.seed(s+1)
    robust_noiseless = (n_1 / (s * np.log(p) + np.log(n_1)))**0.5  # robust parameter for initial estimation
    c_0 = 0.2  # constant for DP robust parameter
    robust = c_0 * (n_2 / (s * np.log(p) + np.log(n_2)))**0.5
    robust_noise = c_0 * (n_2 * mu / (s * np.log(p) + np.log(n_2)))**0.5
    
    robust_low1 = 0.5 * (n_2 * mu / (s + 1 + np.log(n_2)))**0.5
    robust_low2 = 0.5 * (n_2   / (s + 1 + np.log(n_2)))**0.5
    
    result_mse = []  # Store MSE for each fold

    for train_index, test_index in kf.split(X_COMM):  # Split data into 10 folds
        
        # Split the data into training and testing sets for each fold
        X_train, X_test = X_COMM.iloc[train_index], X_COMM.iloc[test_index]
        Y_train, Y_test = Y_COMM.iloc[train_index], Y_COMM.iloc[test_index]
        Y_train = pd.to_numeric(Y_train)
        Y_test = pd.to_numeric(Y_test)

        # Subsample for initial estimation
        random_rows = np.random.choice(len(X_train), size=n_1, replace=False)
        X_subsample = X_train.iloc[random_rows]
        Y_subsample = Y_train.iloc[random_rows]
        
        # Initial model fit with Huber regression
        model_sub = Huber(X_subsample, Y_subsample, intercept=True)
        initial = model_sub.gd_highdim(lr=0.5, T=4, s=s, tau=None, robust=robust_noiseless, beta0=np.array([]), 
                                       standardize=False)
        
        # Fit the model on the remaining data (rest of the data)
        # Use .iloc to ensure correct row selection using integer-based indexing
        X_rest = X_train.drop(index=X_train.index[random_rows])
        Y_rest = Y_train.drop(index=Y_train.index[random_rows])
        beta0 = initial['beta']
        
        model = Huber(X_rest, Y_rest, intercept=True)
        out_Huber_noise_new = model.noisygd_highdim(mu=mu, T=T, delta=delta, lr=lr, beta0=beta0, s=s, 
                                                       robust_low1=robust_low1, robust_low2=robust_low2, robust_high1=robust, 
                                                       robust_high2=robust_noise, B_high=B_huber, standardize=False)
        
        # Predict on the test set
        X_test = np.concatenate([np.ones((len(X_test), 1)), X_test], axis=1)
        Y_test_predict = X_test @ out_Huber_noise_new['beta1']
        
        # Calculate Mean Squared Error (MSE) for this fold
        mse = np.mean((Y_test_predict - Y_test) ** 2)
        result_mse.append(mse)

    # Average the MSE across all folds
    re_matrix[s-1, 0] = np.mean(result_mse)

# Find the best parameter s (with minimum MSE)
min_index = np.argmin(re_matrix[:, 0])
final_re = re_matrix[min_index]
print(f"Optimal s = {min_index + 1}, Final MSE = {final_re}")

Processing s = 1
Processing s = 2
Processing s = 3
Processing s = 4
Processing s = 5
Processing s = 6
Processing s = 7
Processing s = 8
Processing s = 9
Processing s = 10
Optimal s = 3, Final MSE = [0.43433241]


### Comparison of sparse DP Huber and sparse DP LS Methods

Using **s = 3**, estimated through cross-validation, we compare the prediction errors of the **sparse DP Huber**(`noisygd_highdim`) and **sparse DP LS**(`noisygd_ls`) methods. The results from **sparse Huber**(`gd_highdim`,without noise) serve as the **benchmark** for reference, allowing us to evaluate how the introduction of noise and privacy constraints impacts the performance of the other methods.


In [9]:
n = 1994
s = 3
p = X_COMM.shape[1]  
n_1 = 200 
mu = 0.5 
delta = 0.01
lr = 0.1   
robust_noiseless = (n_1/(s*np.log(p)+np.log(n_1)))**0.5
sequence = np.arange(500,1901,100)
result_Huber = []
result_Huber_noise = []
result_ls_noise = []
for index in range(len(sequence)):
    rgt.seed(index+1)
    numm = sequence[index]
    print(numm)
    
    random_rows1 = np.random.choice(n, numm, replace=False)
    Y_COMM_sample = Y_COMM.iloc[random_rows1]
    Y_COMM_sample = Y_COMM_sample.reset_index(drop=True)
    X_COMM_sample = X_COMM.iloc[random_rows1]
    X_COMM_sample = X_COMM_sample.reset_index(drop=True)
    
    n_2 = int(0.8*numm) - n_1 
    B_huber = .4*(np.log(p) + np.log(n_2))**0.5  #  ## truncation parameter 
    T =   int(np.ceil(np.log(n_2)))  ## number of iterations
    robust = .2 * (n_2/(s*np.log(p)+np.log(n_2)))**0.5
    robust_noise = .2 * (n_2*mu/(s*np.log(p)+np.log(n_2)))**0.5 
    robust_low1 = .5*(n_2 /(s+1+np.log(n_2)))**0.5
    robust_low2 = .5*(n_2*mu/(s+1+np.log(n_2)))**0.5
    
    
    M=300 
    HD_Huber = np.zeros(M)
    HD_Huber_noise = np.zeros(M)
    HD_ls_noise = np.zeros(M)
    for rep in list(range(M)):
        rgt.seed(rep+1)
        #print(rep)
        # Split the data into training and test sets (80%-20%)
        train_idx = np.random.choice(numm, size=int(0.8 * numm), replace=False)
        test_idx = np.setdiff1d(np.arange(numm), train_idx)  # Get the remaining 20% as test set
        
        X_train = X_COMM_sample.iloc[train_idx]
        Y_train = Y_COMM_sample.iloc[train_idx]
        X_test = X_COMM_sample.iloc[test_idx]
        Y_test = Y_COMM_sample.iloc[test_idx]
        
        Y_train = pd.to_numeric(Y_train)
        Y_test = pd.to_numeric(Y_test)

        # benchmark
        model_benchmark = Huber(X_train, Y_train, intercept=True)
        benchmark = model_benchmark.gd_highdim(lr=0.5, T=T, s=s, tau=None, robust=robust_noiseless, beta0=np.array([]), 
                                               standardize=False)
        beta_huber = benchmark['beta'] 
        beta_norm = beta_huber.dot(beta_huber)**0.5

        # Subsample for initial estimation  
        random_rows = np.random.choice(len(X_train), size=n_1, replace=False)
        X_subsample = X_train.iloc[random_rows]
        Y_subsample = Y_train.iloc[random_rows]
        Y_subsample = pd.to_numeric(Y_subsample)
        
        model_sub = Huber(X_subsample, Y_subsample, intercept=True)
        initial = model_sub.gd_highdim(lr=0.5, T=4, s=s, tau=None, robust=robust_noiseless, 
                                       beta0=np.array([]),  standardize=False) 

        # Rest for DP Huber  
        X_rest = X_train.drop(index=X_train.index[random_rows])
        Y_rest = Y_train.drop(index=Y_train.index[random_rows])
        beta0 = initial['beta']
        
        model = Huber(X_rest, Y_rest, intercept=True)
        out_Huber_noise_new = model.noisygd_highdim(mu=mu, T=T, delta=delta, lr=lr, beta0=beta0, 
                                                       s=s, robust_low1=robust_low1,robust_low2=robust_low2, robust_high1=robust, 
                                                       robust_high2=robust_noise, B_high=B_huber, standardize=False)
        

         ## Parameters for DP LS 
        n_ls = int(0.8*numm)
        lr_ls = 0.1  ## learning rate for DP LS  
        C_ls = 1.01 * beta_norm # feasibility parameter for DP LS 
        R_ls = 0.1*np.sqrt(2*np.log(n_ls)) 
        c_x = 1
        B_ls = 4*(R_ls+C_ls*c_x)*c_x/np.sqrt(s)
        # Center the features for LS
        X_cent = X_train.to_numpy() - np.mean(X_train.to_numpy(), axis=0)
        Y_cent = Y_train - np.mean(Y_train)
        model_ls = Huber(X_cent, Y_cent, intercept=False)
        out_LS = model_ls.noisygd_ls(mu=mu, T=T, delta=delta, lr=lr_ls, s=s, R=R_ls, C=C_ls, B=B_ls, 
                                    beta0=np.array([]), standardize=False)

        # MSE calculations
        X_test_new = np.concatenate([np.ones((len(X_test), 1)), X_test], axis=1)
        Huber_pre = X_test_new @ beta_huber
        Noise_Huber_pre = X_test_new @ out_Huber_noise_new['beta2']
        
        X_test_cent = X_test.to_numpy() - np.mean(X_test.to_numpy(), axis=0)
        LS_pre = X_test_cent @ out_LS['beta']

        Y_test_cent =  Y_test - np.mean(Y_test)
        HD_Huber[rep] = np.mean((Huber_pre - Y_test) ** 2)
        HD_Huber_noise[rep] = np.mean((Noise_Huber_pre - Y_test) ** 2)
        HD_ls_noise[rep] = np.mean((LS_pre - Y_test_cent) ** 2)

    # Average MSE for this sample size
    result_Huber.append(np.mean(HD_Huber))
    result_Huber_noise.append(np.mean(HD_Huber_noise))
    result_ls_noise.append(np.mean(HD_ls_noise))

500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
